## BRONZE TO SILVER LAYER

### GOLD LAYER - PROCESS HOLDING RECORDS HISTORY


In [15]:
import pandas as pd
from datetime import datetime
from common_utilities import global_path, logger

# Load holdings data 
df = pd.read_csv(global_path.holdings_gold_file_path)
# Convert the datetime column to datetime objects
df["datetime"] = pd.to_datetime(df["datetime"])
# Extract date from datetime for grouping
df["date"] = pd.to_datetime(df["datetime"].dt.date)
# find the max datetime row for each name and date
df = df.sort_values("datetime").drop_duplicates(
    ["stock_name", "date"], keep="last"
)

# Process each stock name separately and fill missing dates
result = []
for stock_name, group in df.groupby("stock_name"):
    # Set date as index and reindex
    group = group.set_index(group["date"])
    # Ensure there are no duplicate dates before reindexing
    date_range = pd.date_range(
        start=group.index.min(),
        end=datetime.today().date(),
        freq="D",
    )
    # Reindex to include all dates and forward fill missing values
    group = group.reindex(date_range, method="ffill").reset_index(drop=True)
    result.append(group)

# Combine processed DataFrames into one
df = pd.concat(result, ignore_index=True)

# Load stock prices and merge with the main DataFrame
df_StockPrice = pd.read_csv(global_path.stockprice_silver_file_path)
df_StockPrice["date"] = pd.to_datetime(df_StockPrice["date"])
logger.info(
    f"Loaded SILVER Layer stock price data from: {global_path.stockprice_silver_file_path}"
)

# Merge stock price data
df = pd.merge(
    df,
    df_StockPrice,
    on=["date", "stock_name"],
    how="left",
)

# Calculate current value, PnL amount, and PnL percentage
df["current_value"] = df["ltp"] * df["holding_quantity"]
df["pnl_amount"] = df["current_value"] - df["holding_amount"]
df["pnl_percentage"] = (df["pnl_amount"] / df["holding_amount"]) * 100
df = df[(df["holding_amount"] != 0) & (df["current_value"] != 0)]
df = df.round(2)

# Final sorting and column selection
df = df.sort_values(by=["date","segment","stock_name"])
df = df[
    [
        "date",
        "segment",
        "stock_name",
        "holding_quantity",
        "holding_price_avg",
        "holding_amount",
        "ltp",
        "current_value",
        "pnl_amount",
        "pnl_percentage",
    ]
]

# Save the final DataFrame to a CSV file
df.to_csv(global_path.holdings_gold_file_path_v2, index=None)
logger.info("GOLD Layer CSV file for Holdings successfully created at:")
logger.info(global_path.holdings_gold_file_path_v2.resolve())

# Display DataFrame information and print success message
df.info()

2024-08-06T01:35:20Z - INFO - Loaded SILVER Layer stock price data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\StockPrice\StockPrice_data.csv
2024-08-06T01:35:20Z - INFO - GOLD Layer CSV file for Holdings successfully created at:
2024-08-06T01:35:20Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\Holdings\Holdings_datav2.csv


<class 'pandas.core.frame.DataFrame'>
Index: 6174 entries, 15245 to 9655
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               6174 non-null   datetime64[ns]
 1   segment            6174 non-null   object        
 2   stock_name         6174 non-null   object        
 3   holding_quantity   6174 non-null   float64       
 4   holding_price_avg  6174 non-null   float64       
 5   holding_amount     6174 non-null   float64       
 6   ltp                5809 non-null   float64       
 7   current_value      5809 non-null   float64       
 8   pnl_amount         5809 non-null   float64       
 9   pnl_percentage     5809 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 530.6+ KB


In [16]:
df[df["date"].dt.date == datetime.today().date()]

,date,segment,stock_name,holding_quantity,holding_price_avg,holding_amount,ltp,current_value,pnl_amount,pnl_percentage


In [17]:
# Read the CSV file
df_trade_history = pd.read_csv(global_path.tradehistory_silver_file_path)

df_trade_history["quantity"] = df_trade_history.apply(
    lambda row: (
        row["quantity"] if row["side"] == "BUY" else row["quantity"] * (-1)
    ),
    axis=1,
)

df_trade_history = (
    df_trade_history.groupby(["exchange", "segment", "stock_name"])["quantity"]
    .sum()
    .reset_index()
)
df_trade_history = df_trade_history[df_trade_history["quantity"] != 0]
df_trade_history = df_trade_history.sort_values(
    by=["exchange", "segment", "stock_name"]
).reset_index(drop=True)
df_trade_history

,exchange,segment,stock_name,quantity
0,BMF,MF,MIRAE-ASSET-TAX-SAVER-DIRECT-GROWTH,39.80
1,BMF,MF,SBI-MAGNUM-TAXGAIN-SCHEME-DIR-GR,4.53
2,FON,FO,NIFTY-CE-24700-08AUG2024,25.00
3,FON,FO,NIFTY-PE-24650-18JUL2024,100.00
4,NSE,EQ,IRCTC,3.00
5,NSE,EQ,KPITTECH,2.00
6,NSE,EQ,TATAMOTORS,100.00
7,NSE,EQ,TATAPOWER,150.00
